# Why statefulness is key

### Form filling
You'll often want your bot to guide users through a series of steps, such as when they're placing an order.

In this exercise, you'll begin building a bot that lets users order coffee. They can choose between two types: Colombian and Kenyan. If the user provides unexpected input, your bot will handle this differently depending on where they are in the flow.

Your job here is to identify the appropriate state and next state based on the intents and response messages provided. For example, if the intent is "order", then the state changes from INIT to CHOOSE_COFFEE.

A function send_message(policy, state, message) has already been defined for you. It takes the policy, the current state, and message as arguments, and returns the new state as a result. Additionally, an interpret(message) function, similar to the one Alan described in the video, has been pre-defined for you.

In [1]:
# Define the INIT state
INIT = 0
# Define the CHOOSE_COFFEE state
CHOOSE_COFFEE = 1

# Define the ORDERED state
ORDERED = 2

# Define the policy rules
policy = {
    (INIT, "order"): (CHOOSE_COFFEE, "ok, Colombian or Kenyan?"),
    (INIT, "none"): (INIT, "I'm sorry - I'm not sure how to help you"),
    (CHOOSE_COFFEE, "specify_coffee"): (ORDERED, "perfect, the beans are on their way!"),
    (CHOOSE_COFFEE, "none"): (CHOOSE_COFFEE, "I'm sorry - would you like Colombian or Kenyan?"),
}

# Create the list of messages
messages = [
    "I'd like to become a professional dancer",
    "well then I'd like to order some coffee",
    "my favourite animal is a zebra",
    "kenyan"
]

# Call send_message() for each message
state = INIT
for message in messages:    
    state = send_message(policy,state,message)

NameError: name 'send_message' is not defined

### Asking contextual questions
Sometimes your users need some help! They will have questions and expect the bot to help them.

In this exercise, you'll allow users to ask the coffee bot to explain the steps to them. As in the previous exercise, the answer they get will depend on where they are in the flow.

In [2]:

# Define the states
INIT=0 
CHOOSE_COFFEE=1
ORDERED=2

# Define the policy rules dictionary
policy_rules = {
    (INIT, "ask_explanation"): (INIT, "I'm a bot to help you order coffee beans"),
    (INIT, "order"): (CHOOSE_COFFEE, "ok, Colombian or Kenyan?"),
    (CHOOSE_COFFEE, "specify_coffee"): (ORDERED, "perfect, the beans are on their way!"),
    (CHOOSE_COFFEE, "ask_explanation"): (CHOOSE_COFFEE, "We have two kinds of coffee beans - the Kenyan ones make a slightly sweeter coffee, and cost $6. The Brazilian beans make a nutty coffee and cost $5.")    
}

# Define send_messages()
def send_messages(messages):
    state = INIT
    for msg in messages:
        state = send_message(state,msg)

# Send the messages
send_messages([
    "what can you do for me?",
    "well then I'd like to order some coffee",
    "what do you mean by that?",
    "kenyan"
])

NameError: name 'send_message' is not defined

### Dealing with rejection
What happens if you make a suggestion to your user and they don't like it? Your bot will look really silly if it makes the same suggestion again right away.

Here, you're going to modify your respond() function so that it accepts and returns 4 arguments:

The user message as an argument, and the bot response as the first return value.
A dictionary params including the entities the user has specified.
A prev_suggestions list. When passed to respond(), this should contain the suggestions made in the previous bot message. When returned by respond(), it should contain the current suggestions.
An excluded list, which contains all of the results your user has already explicitly rejected.
Your function should add the previous suggestions to the excluded list whenever it receives a "deny" intent. It should also filter out excluded suggestions from the response.

In [3]:

# Define respond()
def respond(message,params,prev_suggestions,excluded):
    # Interpret the message
    parse_data = interpret(message)
    # Extract the intent
    intent = parse_data["intent"]["name"]
    # Extract the entities
    entities = parse_data["entities"]
    # Add the suggestion to the excluded list if intent is "deny"
    if intent == "deny":
        excluded.extend(prev_suggestions)
    # Fill the dictionary with entities	
    for ent in entities:
        params[ent["entity"]] = str(ent["value"])
    # Find matching hotels
    results = [
        r 
        for r in find_hotels(params, excluded) 
        if r[0] not in excluded
    ]
    # Extract the suggestions
    names = [r[0] for r in results]
    n = min(len(results), 3)
    suggestions = names[:2]
    return responses[n].format(*names), params, suggestions, excluded

# Initialize the empty dictionary and lists
params, suggestions, excluded = {}, [], []

# Send the messages
for message in ["I want a mid range hotel", "no that doesn't work for me"]:
    print("USER: {}".format(message))
    response, params, suggestions, excluded = respond(message, params, suggestions, excluded)
    print("BOT: {}".format(response))

USER: I want a mid range hotel


NameError: name 'interpret' is not defined

# Asking Questions and Queuing Answers

### Pending actions I
You can really improve the user experience of your bot by asking the user simple yes or no follow-up questions. One easy way to handle these follow-ups is to define pending actions which get executed as soon as the user says "yes", and wiped if the user says "no".

In this exercise, you're going to define a policy() function which takes the intent as its sole argument and returns two values: The next action to take and a pending action. The policy function should return this pending action when a "yes" or "affirm" intent is returned and should wipe the pending actions if a "no" or "deny" intent is returned.

Here, the interpret(message) function has been defined for you such that if "yes" is in the message, "affirm" is returned, and if "no" is in the message, then "deny" is returned.

In [4]:
# Define policy()
def policy(intent):
    # Return "do_pending" if the intent is "affirm"
    if intent == "affirm":
        return "do_pending", None
    # Return "Ok" if the intent is "deny"
    if intent == "deny":
        return "Ok", None
    if intent == "order":
        return "Unfortunately, the Kenyan coffee is currently out of stock, would you like to order the Brazilian beans?", "Alright, I've ordered that for you!"

### Pending actions II
Having defined your policy() function, it's now time to write a send_message() function which takes both a pending action and a message as its arguments and leverages the policy() function to determine the bot's response.

Your policy(intent) function from the previous exercise has been pre-loaded.

In [5]:
# Define send_message()
def send_message(pending, message):
    print("USER : {}".format(message))
    action,pending_action = policy(interpret(message))
    if action == "do_pending" and pending is not None:
        print("BOT : {}".format(pending))
    else:
        print("BOT : {}".format(action))
    return pending_action
    
# Define send_messages()
def send_messages(messages):
    pending = None
    for msg in messages:
        pending = send_message(pending,msg)

# Send the messages
send_messages([
    "I'd like to order some coffee",
    "ok yes please"
])

USER : I'd like to order some coffee


NameError: name 'interpret' is not defined

### Pending state transitions
You'll often need to briefly deviate from the flow of a conversation, for example to authenticate a user, before returning to the topic of discussion.

In these cases, it's often simpler - and easier to debug - if you save some actions/states as pending rather than adding ever more complicated rules.

Here, you're going to define a policy_rules dictionary, where the keys are tuples of the current state and the received intent, and the values are tuples of the next state, the bot's response, and a state for which to set a pending transition.

In [7]:
# Define the states
INIT=0
AUTHED=1
CHOOSE_COFFEE=2
ORDERED=3

# Define the policy rules
policy_rules = {
    (INIT, "order"): (INIT, "you'll have to log in first, what's your phone number?", AUTHED),
    (INIT, "number"): (AUTHED, "perfect, welcome back!", None),
    (AUTHED, "order"): (CHOOSE_COFFEE, "would you like Colombian or Kenyan?", None),    
    (CHOOSE_COFFEE, "specify_coffee"): (ORDERED, "perfect, the beans are on their way!", None)
}

# Define send_messages()
def send_messages(messages):
    state = INIT
    pending = None
    for msg in messages:
        state, pending = send_message(state, pending, msg)

# Send the messages
send_messages([
    "I'd like to order some coffee",
    "555-1234",
    "kenyan"
])

TypeError: send_message() takes 2 positional arguments but 3 were given

### Putting it all together I
It's time to put everything you've learned in the course together by combining the coffee ordering bot with the ELIZA rules from chapter 1.

To begin, you'll define a function called chitchat_response(), which calls the predefined function match_rule() from back in chapter 1. This returns a response if the message matched an ELIZA template, and otherwise, None.

The ELIZA rules are contained in a dictionary called eliza_rules.

In [8]:
# Define chitchat_response()
def chitchat_response(message):
    # Call match_rule()
    response, phrase = match_rule(eliza_rules,message)
    # Return none if response is "default"
    if response == "default":
        return None
    if '{0}' in response:
        # Replace the pronouns of phrase
        phrase = replace_pronouns(phrase)
        # Calculate the response
        response = response.format(phrase)
    return response

### Putting it all together II
With your chitchat_response(message) function defined, the next step is to define a send_message() function. This function should first call chitchat_response(message) and only use the coffee bot policy if there is no matching message.

In [9]:
# Define send_message()
def send_message(state,pending,message):
    print("USER : {}".format(message))
    response = chitchat_response(message)
    if response is not None:
        print("BOT : {}".format(response))
        return state, None
    
    # Calculate the new_state, response, and pending_state
    new_state, response, pending_state = policy_rules[(state, interpret(message))]
    print("BOT : {}".format(response))
    if pending is not None:
        new_state, response, pending_state = policy_rules[pending]
        print("BOT : {}".format(response))        
    if pending_state is not None:
        pending = (pending_state, interpret(message))
    return new_state, pending

# Define send_messages()
def send_messages(messages):
    state = INIT
    pending = None
    for msg in messages:
        state, pending = send_message(state, pending, msg)

# Send the messages
send_messages([
    "I'd like to order some coffee",
    "555-12345",
    "do you remember when I ordered 1000 kilos by accident?",
    "kenyan"
])  

USER : I'd like to order some coffee


NameError: name 'match_rule' is not defined

# Frontiers of Dialogue Technology

### Generating text with neural networks
In this final exercise of the course, you're going to generate text using a neural network trained on the scripts of every episode of The Simpsons. Specifically, you'll use a simplified version of the sample_text() function that Alan described in the video.

It takes in two arguments: seed and temperature. The seed argument is the initial sequence that the network uses to generate the subsequent text, while the temperature argument controls how risky the network is when generating text. At very low temperatures, it just repeats the most common combinations of letters, and at very high temperatures, it generates complete gibberish. In order to ensure fast runtimes, the network in this exercise will only work for a subset of temperature values.

In [10]:
# Feed the seed text into the neural network
seed = "i'm gonna punch lenny in the back of the"

# Iterate over the different temperature values
for temperature in [0.2, 0.5, 1.0, 1.2]:
    print("\nGenerating text with riskiness : {}\n".format(temperature))
    # Call the sample_text function
    print(sample_text(seed,temperature))


Generating text with riskiness : 0.2



NameError: name 'sample_text' is not defined